## Assignment 1 - Task 1
### This Jupyter file exists for the purposes of cleaning and loading in all required datasets into an SQL database

In [16]:
"""
Initial DB Connection And Library Loading
"""
from sqlalchemy import create_engine
import psycopg2
import psycopg2.extras
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon
from geoalchemy2 import Geometry, WKTElement
import matplotlib.pyplot as plt
from shapely.wkt import loads
from sqlalchemy import text

credentials = "Credentials.json"

def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict["host"]
        db_user    = db_conn_dict["user"]
        db_pw      = db_conn_dict["password"]
        default_db = db_conn_dict["user"]
        port       = db_conn_dict["port"]
        try:
            db = create_engine(f"postgresql+psycopg2://{db_user}:{db_pw}@{host}:{port}/{default_db}", echo=False)
            conn = db.connect()
            print("Connected successfully.")
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db,conn

def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(text(sqlcmd), args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep="\n")
    return result

In [61]:
db, conn = pgconnect(credentials)

Connected successfully.


In [18]:
"""
Loading In Datasets
"""

# Loading in SA2 dataset
sa2 = gpd.read_file("SA2/SA2_2021_AUST_GDA2020.shp")

# Loading in school dataset
future = gpd.read_file("catchments/catchments_future.shp")
primary = gpd.read_file("catchments/catchments_primary.shp")
secondary = gpd.read_file("catchments/catchments_secondary.shp")

# Loading in Businesses
businesses = pd.read_csv("Businesses.csv")

# Loading in Income
income = pd.read_csv("Income.csv")

# Loading in Polling Places
polling = pd.read_csv("PollingPlaces2019.csv")

# Loading in Populations
population = pd.read_csv("Population.csv")

# Loading in Stops
stop = pd.read_csv("Stops.txt")

In [19]:
"""
Data Cleaning
"""


"""
Helper Functions
"""
def lowercase_column_names(df):
    return df.rename(columns=lambda x: x.lower())

srid = 4326  # The chosen SRID for all datasets (the international SRID)
def create_wkt_element(geom, srid):
    if geom.geom_type == "Polygon":
        geom = MultiPolygon([geom])
    return WKTElement(geom.wkt, srid)
"""
END OF Helper Functions
"""


# Turning all column names to lowercase
sa2 = lowercase_column_names(sa2)
future = lowercase_column_names(future)
primary = lowercase_column_names(primary)
secondary = lowercase_column_names(secondary)
businesses = lowercase_column_names(businesses)
income = lowercase_column_names(income)
polling = lowercase_column_names(polling)
population = lowercase_column_names(population)
stop = lowercase_column_names(stop)


"""
sa2 File
"""
# Removing rows with invalid polygons
sa2 = sa2[sa2.geometry.is_valid]

# Removing rows where the GCC isn"t "Greater Sydney"
sa2 = sa2[sa2["gcc_name21"] == "Greater Sydney"]

# Polygons turned into multipolygons
sa2["geom"] = sa2["geometry"].apply(lambda x: create_wkt_element(geom=x,srid=srid))
sa2 = sa2.drop(columns="geometry")

# Renaming "sa2_code21" to "sa2_code"
sa2 = sa2.rename(columns={"sa2_code21": "sa2_code"})

# Renaming "sa2_name21" to "sa2_name"
sa2 = sa2.rename(columns={"sa2_name21": "sa2_name"})


"""
Catchment Files
"""
# Adding the priority column to the future table
future.insert(future.columns.get_loc("year12") + 1, "priority", "None")

# Changing all instances of "2024", "2025", "2026" and "2027" to "Y" and all instances of "0" to "N"
columns_to_replace = ["kindergart", "year1", "year2", "year3", "year4", "year5", "year6", "year7", "year8", "year9", "year10", "year11", "year12"]
future[columns_to_replace] = future[columns_to_replace].replace({2024: "Y", 2025: "Y", 2026: "Y", 2027: "Y", 0: "N"})

# Polygons turned into multipolygons
future["geom"] = future["geometry"].apply(lambda x: create_wkt_element(geom=x,srid=srid))
future = future.drop(columns="geometry")

primary["geom"] = primary["geometry"].apply(lambda x: create_wkt_element(geom=x,srid=srid))
primary = primary.drop(columns="geometry")

secondary["geom"] = secondary["geometry"].apply(lambda x: create_wkt_element(geom=x,srid=srid))
secondary = secondary.drop(columns="geometry")

# Joining "future", "primary" and "secondary" into one table namely "school"
school = pd.concat([future, primary, secondary], ignore_index=True)

# Turning the "add_date" column into a datetime datatype
school["add_date"] = pd.to_datetime(school["add_date"], format="%Y%m%d", errors="coerce")


"""
Business File
"""
# Recalculating the total_businesses column (some rows had been incorrectly summed)
business_sum = businesses.iloc[:, 4:9].sum(axis=1)
businesses["total_businesses"] = business_sum


"""
Income File
"""
# Removes any row whose column has "np"
income = income[income.apply(lambda row: "np" not in row.values, axis=1)]

# Renaming "sa2_code21" to "sa2_code"
income = income.rename(columns={"sa2_code21": "sa2_code"})

"""
Polling File
"""
# Removing all rows where the geom data is "NaN"
polling = polling.dropna(subset=["the_geom"])

# Renaming "the_geom" to "geom"
polling = polling.rename(columns={"the_geom": "geom"})

# Conversion to WKT format
polling["geom"] = polling["geom"].apply(lambda x: WKTElement(loads(x).wkt, srid=srid))


"""
Population File
"""
# Nothing modified in the population csv


"""
Stops File
"""
# Converting lat and long coordinates to a point object
stop["geom"] = gpd.points_from_xy(stop.stop_lat, stop.stop_lon)
stop["geom"] = stop["geom"].apply(lambda x: WKTElement(x.wkt, srid=srid))

In [20]:
"""
Table Creation
"""

# Creating the SA2 table
conn.execute("""
DROP TABLE IF EXISTS sa2;
CREATE TABLE sa2 (
    sa2_code NUMERIC, 
    sa2_name TEXT, 
    chg_flag21 NUMERIC, 
    chg_lbl21 TEXT, 
    sa3_code21 NUMERIC,
    sa3_name21 TEXT,
    sa4_code21 NUMERIC,
    sa4_name21 TEXT,
    gcc_code21 TEXT,
    gcc_name21 TEXT,
    ste_code21 NUMERIC,
    ste_name21 TEXT,
    aus_code21 TEXT,
    aus_name21 TEXT,
    areasqkm21 DOUBLE PRECISION,
    loci_uri21 TEXT,
    geom GEOMETRY(MULTIPOLYGON,4326),
    PRIMARY KEY (sa2_code)
);"""
)

# Creating the school table
conn.execute("""
DROP TABLE IF EXISTS school;
CREATE TABLE school (
    use_id NUMERIC,
    catch_type TEXT,
    use_desc TEXT,
    add_date TIMESTAMP,
    kindergart TEXT,
    year1 TEXT,
    year2 TEXT,
    year3 TEXT,
    year4 TEXT,
    year5 TEXT,
    year6 TEXT,
    year7 TEXT,
    year8 TEXT,
    year9 TEXT,
    year10 TEXT,
    year11 TEXT,
    year12 TEXT,
    priority TEXT,
    geom GEOMETRY(MULTIPOLYGON,4326),
    PRIMARY KEY (use_id, catch_type)
);"""
)

# Creating the businesses table
conn.execute("""
DROP TABLE IF EXISTS businesses;
CREATE TABLE businesses (
    industry_code TEXT,
    industry_name TEXT,
    sa2_code NUMERIC,
    sa2_name TEXT,
    "0_to_50k_businesses" NUMERIC,
    "50k_to_200k_businesses" NUMERIC,
    "200k_to_2m_businesses" NUMERIC,
    "2m_to_5m_businesses" NUMERIC,
    "5m_to_10m_businesses" NUMERIC,
    "10m_or_more_businesses" NUMERIC,
    total_businesses NUMERIC,
    PRIMARY KEY (industry_name, sa2_code)
);"""
)

# Creating the income table
conn.execute("""
DROP TABLE IF EXISTS income;
CREATE TABLE income (
    sa2_code NUMERIC,
    sa2_name TEXT,
    earners NUMERIC,
    median_age NUMERIC,
    median_income NUMERIC,
    mean_income NUMERIC,
    PRIMARY KEY (sa2_code)
);"""
)

# Creating the polling table
conn.execute("""
DROP TABLE IF EXISTS polling;
CREATE TABLE polling (
    fid TEXT,
    state TEXT,
    division_id NUMERIC,
    division_name TEXT,
    polling_place_id NUMERIC,
    polling_place_type_id NUMERIC,
    polling_place_name TEXT,
    premises_name TEXT,
    premises_address_1 TEXT,
    premises_address_2 TEXT,
    premises_address_3 TEXT,
    premises_suburb TEXT,
    premises_state_abbreviation TEXT,
    premises_postal_code NUMERIC,
    latitude DOUBLE PRECISION,
    longitude DOUBLE PRECISION,
    geom GEOMETRY(POINT,4326),
    PRIMARY KEY (fid)
);"""
)

# Creating the population table
conn.execute("""
DROP TABLE IF EXISTS population;
CREATE TABLE population (
    sa2_code NUMERIC,
    sa2_name TEXT,
    "0-4_people" NUMERIC,
    "5-9_people" NUMERIC,
    "10-14_people" NUMERIC,
    "15-19_people" NUMERIC,
    "20-24_people" NUMERIC,
    "25-29_people" NUMERIC,
    "30-34_people" NUMERIC,
    "35-39_people" NUMERIC,
    "40-44_people" NUMERIC,
    "45-49_people" NUMERIC,
    "50-54_people" NUMERIC,
    "55-59_people" NUMERIC,
    "60-64_people" NUMERIC,
    "65-69_people" NUMERIC,
    "70-74_people" NUMERIC,
    "75-79_people" NUMERIC,
    "80-84_people" NUMERIC,
    "85-and-over_people" NUMERIC,
    total_people NUMERIC,
    PRIMARY KEY (sa2_code)
);"""
)

# Creating the stop table
conn.execute("""
DROP TABLE IF EXISTS stop;
CREATE TABLE stop (
    stop_id NUMERIC,
    stop_code NUMERIC,
    stop_name TEXT,
    stop_lat DOUBLE PRECISION,
    stop_long DOUBLE PRECISION,
    location_type NUMERIC,
    parent_station NUMERIC,
    wheelchair_boarding NUMERIC,
    platform_code NUMERIC,
    geom GEOMETRY(POINT,4326),
    PRIMARY KEY (stop_id)
);"""
)

ObjectNotExecutableError: Not an executable object: '\nDROP TABLE IF EXISTS sa2;\nCREATE TABLE sa2 (\n    sa2_code NUMERIC, \n    sa2_name TEXT, \n    chg_flag21 NUMERIC, \n    chg_lbl21 TEXT, \n    sa3_code21 NUMERIC,\n    sa3_name21 TEXT,\n    sa4_code21 NUMERIC,\n    sa4_name21 TEXT,\n    gcc_code21 TEXT,\n    gcc_name21 TEXT,\n    ste_code21 NUMERIC,\n    ste_name21 TEXT,\n    aus_code21 TEXT,\n    aus_name21 TEXT,\n    areasqkm21 DOUBLE PRECISION,\n    loci_uri21 TEXT,\n    geom GEOMETRY(MULTIPOLYGON,4326),\n    PRIMARY KEY (sa2_code)\n);'

In [49]:
"""
Inserting Into SQL Tables
"""

sa2.to_sql("sa2", conn, if_exists="replace", index=False, dtype={"geom": Geometry("MULTIPOLYGON", srid)})
school.to_sql("school", conn, if_exists="replace", index=False, dtype={"geom": Geometry("MULTIPOLYGON", srid)})
businesses.to_sql("businesses", conn, if_exists="replace", index=False)
income.to_sql("income", conn, if_exists="replace", index=False)
polling.to_sql("polling", conn, if_exists="replace", index=False, dtype={"geom": Geometry("POINT", srid)})
population.to_sql("population", conn, if_exists="replace", index=False)
stop.to_sql("stop", conn, if_exists="replace", index=False, dtype={"geom": Geometry("POINT", srid)})

718

In [62]:
"""
JUST TO TEST SQL QUERIES, REMOVE LATER
"""

sql = """
SELECT *
FROM income
"""
query(conn, sql)

,sa2_code,sa2_name,earners,median_age,median_income,mean_income
0,101021007,Braidwood,2467,51,46640,68904
1,101021008,Karabar,5103,42,65564,69672
2,101021009,Queanbeyan,7028,39,63528,69174
3,101021010,Queanbeyan - East,3398,39,66148,74162
4,101021012,Queanbeyan West - Jerrabomberra,8422,44,78630,91981
...,...,...,...,...,...,...
1900,128021537,Royal National Park,14,37,36980,47584
1901,128021538,Sutherland - Kirrawee,13895,41,64940,74867
1902,128021607,Engadine,10239,43,63695,72995
1903,128021608,Loftus - Yarrawarrah,4424,45,63087,76440


In [63]:
conn.close()
db.dispose()